In [ ]:
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand, randn
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot

In [ ]:
def def_dis(n_inputs=2):
  model=Sequential()
  model.add(Dense(25,activation='relu',kernel_initializer='he_uniform',input_dim=n_inputs))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
  return model

In [ ]:
def def_gen(latent_dim,n_outputs=2):
  model=Sequential()
  model.add(Dense(15,activation='relu',kernel_initializer='he_uniform',input_dim=latent_dim))
  model.add(Dense(n_outputs,activation='linear'))
  return model

In [ ]:
def def_gan(generator,discriminator):
  discriminator.trainable=False
  model=Sequential()
  model.add(generator)
  model.add(discriminator)
  model.compile(loss='binary_crossentropy',optimizer='adam')
  return model

In [ ]:
def gen_real_sample(n):
  X1 = rand(n) - 0.5

  X2 = X1 * X1
# stack arrays
  X1 = X1.reshape(n, 1)
  X2 = X2.reshape(n, 1)
  X = hstack((X1, X2))
# generate class labels
  y = ones((n, 1))
  return X, y 
def gen_latent_points(latent_dim,n):
   x_input = randn(latent_dim * n)
# reshape into a batch of inputs for the network
   x_input = x_input.reshape(n, latent_dim)
   return x_input
def gen_fake_sample(generator,latent_dim,n):
  x_in=  gen_latent_points(latent_dim,n)
  x=generator.predict(x_in)
  y=zeros((n,1))
  return x,y

In [ ]:
def summarize_performance(epoch, generator, discriminator, latent_dim, n=100):
       x_re,y_re=gen_real_sample(n)
       _,acc_real=discriminator.evaluate(x_re,y_re,verbose=0)
       x_fake, y_fake = gen_fake_sample(generator, latent_dim, n)
# evaluate discriminator on fake examples
       _, acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0) 
       print(epoch,acc_real,acc_fake)
       pyplot.scatter(x_re[:,0],x_re[:,1],color='red')
       pyplot.scatter(x_fake[:,0],x_fake[:,1],color='blue')
       filename='generated_plot_e%03d.png'%(epoch+1)
       pyplot.savefig(filename)
       pyplot.close()

In [ ]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, latent_dim, n_epochs=10000, n_batch=128,
n_eval=2000):
# determine half the size of one batch, for updating the discriminator
     half_batch = int(n_batch / 2)
     for i in range(n_epochs):
       x_real, y_real = gen_real_sample(half_batch)
       x_fake, y_fake = gen_fake_sample(g_model, latent_dim, half_batch)
       d_model.train_on_batch(x_real,y_real)
       d_model.train_on_batch(x_fake,y_fake)

       x_gan=gen_latent_points(latent_dim,n_batch)
       y_gan=ones((n_batch,1))

       gan_model.train_on_batch(x_gan,y_gan)
       if (i+1) % n_eval == 0:
           summarize_performance(i, g_model, d_model, latent_dim)

latent_dim = 5
# create the discriminator
discriminator = def_dis()
# create the generator
generator = def_gen(latent_dim)
# create the gan
gan_model = def_gan(generator, discriminator)
# train model
train(generator, discriminator, gan_model, latent_dim)

1999 0.4000000059604645 0.3700000047683716
3999 0.0 1.0
5999 0.6399999856948853 0.5299999713897705
7999 0.5099999904632568 0.5299999713897705
9999 0.41999998688697815 0.5600000023841858
